# Appendix A: MADBの前処理

`madb`から必要なデータを抽出し，扱いやすいようcsvに変換

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import glob
import json
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import zipfile

In [3]:
DIR_IN = '../madb/data/json-ld'
DIR_TMP = '../data/preprocess/tmp'
DIR_OUT = '../data/preprocess/out'

In [4]:
FNS_CM = [
    'cm102',
    'cm105',
    'cm106',
]

In [5]:
# CM105で使用するカラム
COL_CM105 = [
    'identifier',
    'label',
    'note',
    'publicationPeriodicity',
    'name',
    'publisher',
    'dayPublishedFinal',
]

In [6]:
!ls {DIR_IN}

metadata_an-col_an207_json.zip	 metadata_cm-item_cm101_json.zip
metadata_an-col_an208_json.zip	 metadata_cm-item_cm102_json.zip
metadata_an-col_an210_json.zip	 metadata_cm-item_cm103_json.zip
metadata_an-item_an201_json.zip  metadata_co-curate_co504_json.zip
metadata_an-item_an202_json.zip  metadata_gm-col_gm305_json.zip
metadata_an-item_an205_json.zip  metadata_gm-col_gm306_json.zip
metadata_cm-col_cm104_json.zip	 metadata_gm-item_gm301_json.zip
metadata_cm-col_cm105_json.zip	 metadata_ma-col_ma408_json.zip
metadata_cm-col_cm106_json.zip	 metadata_ma-item_ma401_json.zip


## 関数

In [7]:
def read_json(path):
    """
    jsonファイルを辞書として読み込む関数．

    Params:
        path (str): 読込対象ファイルパス
    Returns:
        dict: 辞書
    """
    with open(path, 'r') as f:
        dct = json.load(f)

    return dct

In [35]:
def save_json(path, dct):
    """
    辞書をjson形式で保存する関数．

    Params:
        path (str): jsonファイルの保存先
        dct (dict): 保存対象辞書
    """
    with open(path, 'w') as f:
        json.dump(dct, f, ensure_ascii=False, indent=4)

## 解凍

マンガ系のデータ（`*cm*`）のみ`DIR_TMP`に解凍する．

In [22]:
ps_cm = glob.glob(f'{DIR_IN}/*_cm*')

In [23]:
for p_from in tqdm(ps_cm):
    p_to = p_from.replace(DIR_IN, DIR_TMP).replace('.zip', '')
    
    with zipfile.ZipFile(p_from) as z:
        z.extractall(p_to)

  0%|          | 0/6 [00:00<?, ?it/s]

## 前処理

### 対象

In [9]:
ps_cm = {cm: glob.glob(f'{DIR_TMP}/*{cm}*/*') for cm in FNS_CM}

In [10]:
ps_cm

{'cm102': ['../data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00001.json',
  '../data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00002.json'],
 'cm105': ['../data/preprocess/tmp/metadata_cm-col_cm105_json/metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'],
 'cm106': ['../data/preprocess/tmp/metadata_cm-col_cm106_json/metadata_cm-col_cm106_json\\metadata_cm-col_cm106_00001.json']}

### `cm105`

In [11]:
cm105 = read_json(ps_cm['cm105'][0])

In [12]:
df_cm105 = pd.DataFrame(cm105['@graph'])[COL_CM105]

In [26]:
def get_published_name(name):
    """nameからpublished_nameを取得"""
    for x in name:
        if type(x) is str:
            return x
    raise Exception(f'No published_name in {name}!')

In [31]:
# 雑誌名を取得
df_cm105['mcname'] = df_cm105['name'].apply(lambda x: get_published_name(x))

In [32]:
# 例
df_cm105.head(3).T

,0,1,2
identifier,C117451,C117452,C117453
label,斬鬼,月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL,YOUNG YOU Special
note,・『ヤングキングSPECIAL増刊 斬鬼』から独立創刊。 ／ ・通巻号数は『ヤングキングSP...,NaN,[編]集英社 ／ 「Young you」の増刊
publicationPeriodicity,隔月刊,不定期刊 （no determinable frequency）,NaN
name,"[斬鬼, {'@language': 'ja-hrkt', '@value': 'ザンキ'}]","[月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL, {'@lan...","[{'@language': 'ja-hrkt', '@value': 'ヤングユースペシャ..."
publisher,少年画報社,集英社 ∥ シュウエイシャ,集英社
dayPublishedFinal,NaN,NaN,NaN
mcname,斬鬼,月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL,YOUNG YOU Special


In [34]:
# mcnameからmcidを引く辞書
mcname2mcid = df_cm105.groupby('mcname')['identifier'].first().to_dict()

In [36]:
# 保存
save_json(os.path.join(DIR_TMP, 'mcname2mcid.json'), mcname2mcid)

### `cm102`

In [38]:
cm102 = read_json(ps_cm['cm102'][0])

In [54]:
def get_items_by_genre(graph, genre):
    """graphから所定のgenreのアイテム群を取得"""
    items = [
        x for x in graph 
        if 'genre' in x.keys() and x['genre'] == genre]
    return items

In [61]:
# 各ジャンルのアイテム群を取得
mis = get_items_by_genre(cm102['@graph'], '雑誌巻号')
eps = get_items_by_genre(cm102['@graph'], 'マンガ作品')

In [59]:
len(mis)

100000

In [63]:
pd.DataFrame(mis).head(3).T

,0,1,2
@id,https://mediaarts-db.bunka.go.jp/id/M533119,https://mediaarts-db.bunka.go.jp/id/M533120,https://mediaarts-db.bunka.go.jp/id/M533121
rdf:type,class:Item,class:Item,class:Item
label,マーガレット 2004年 表示号数13,マーガレット 2004年 表示号数12,マーガレット 2004年 表示号数11
identifier,M533119,M533120,M533121
additionalType,class:CM,class:CM,class:CM
genre,雑誌巻号,雑誌巻号,雑誌巻号
oldDateModified,2016-04-20T20:25:10,2016-04-20T20:25:10,2016-04-20T20:25:10
mid,MARGrgl20040620,MARGrgl20040605,MARGrgl20040520
dayDisplayed,20,5,20
issueNumberDisplayed,13,12,11


In [60]:
len(eps)

119771

In [64]:
pd.DataFrame(eps).head(3).T

,0,1,2
@id,_:b10,_:b1000,_:b10000
genre,マンガ作品,マンガ作品,マンガ作品
oldDateModified,2015-02-24T09:54:48,2015-02-24T06:33:58,2015-02-12T11:26:39
mcid,002,037,005
note,4色カラー,NaN,NaN
pageExclude,0.0,0.0,0.0
relatedCollection,https://mediaarts-db.bunka.go.jp/id/C108614,https://mediaarts-db.bunka.go.jp/id/C106366,https://mediaarts-db.bunka.go.jp/id/C93096
alternativeHeadline,第37回,NaN,第七話 青春とは鎖につながれないことである①
creator,"[篠塚ひろむ, {'@language': 'ja-hrkt', '@value': 'シノ...","[{'@language': 'ja-hrkt', '@value': 'フクベトモミ'},...","[{'@language': 'ja-hrkt', '@value': 'ヤマサキヒロミ ..."
isPartOf,https://mediaarts-db.bunka.go.jp/id/M617830,https://mediaarts-db.bunka.go.jp/id/M617736,https://mediaarts-db.bunka.go.jp/id/M578854


misとepsで必要な情報だけ残して，あとはマージすればOK